# Glossary Generator

即兴想起来就写了，主要目的是配合英语学习的透析阅读法使用，什么叫透析阅读法请自行搜索。

程序比较简单，简单地说，就是：
1. 读取一本小说的文本，干掉复数、时态这些东西，得到一本小说的词汇表；
2. 去掉认识和不常用的词，生成不认识且高频的单词表。

生词表生成后可导入欧陆词典一类的app，快速预习一下，可以大幅提升阅读原版书籍时的体验。

In [ ]:
import os, nltk
import string
import collections
import textract

from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

## 1. 读取文档
目前支持txt/pdf/doc/docx/csv/epub格式，扫描版的PDF不行，只能是文字版的。

除了txt外，其他格式读取时花的时间会长一点，请耐心等待。

In [ ]:
filename = 'pride_and_prejudice.txt' 

读取文本，去掉标点符号。

In [ ]:
byte = textract.process(filename)
text = byte.decode("utf-8").replace('\n', ' ')
text = text.replace('—', '').replace('.', '')
text.translate(str.maketrans('', '', string.punctuation))

## 2. 切割和统计单词
获取单词
抽取词干或词根形式且转成小写
获取排名在1000以后且频率在3以上的单词


In [ ]:
nltk_tokens = nltk.word_tokenize(text) # 切割成单词
words = [porter_stemmer.stem(w) for w in nltk_tokens] # 抽取词的词干或词根形式且转成小写

In [ ]:
word_count = collections.Counter(words)
word_count = sorted(word_count.items(), key=lambda pair: pair[1], reverse=True)

In [ ]:
clean_words = [pair[0] for pair in word_count[1001:] if pair[1]>=3]
len(clean_words)

## 3. 去除熟悉单词

- 柯林斯词典词频: Collins的词库里只有14000-15000单词，但我测试的效果似乎比COCA好一些。大家可以自己更改文件名，看看自己哪一个更合适。
- 自已建立熟词库:和自

In [ ]:
dict_filename = "my_vocabulary.txt" if os.path.exists('my_vocabulary.txt') else 'common30k.txt'

dict_words = open(dict_filename, 'r', encoding='utf-8').read().replace('\n', ' ').split(' ')
if dict_filename=='common30k.txt':
    dict_words = dict_words[8001:]


我附的高频词表是顺序的，即高频词在前，低频词在后。比对时我去掉了前面的8000常用词，如果你觉得自己词汇量低，可以考虑只去掉5000；反之亦然。

In [ ]:
new_words = [w for w in clean_words if w in dict_words]
len(new_words)

这个词表里肯定有很多还是你认识的，不过我觉得在生词本里过一遍（尤其是欧陆提供很多例句）看看怎么用还是有帮助的。如果认识的词特别多，请返回上面一步干掉更多高频词。我个人的小经验是阅读生词量在10%左右的小说学习+阅读体验比较好。

也可以考虑使用其他词频库来对比，比如Collins词频表。

In [ ]:
glossary = filename.split('.')[0] + '_glossary.txt'
with open(glossary, 'w') as output:
    output.write('\n'.join(new_words))

## 4. 将已经背熟的单词加入熟词库